In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [3]:
 df=pd.read_csv("project l2.csv")

In [4]:
 df.head

<bound method NDFrame.head of        Ind_ID GENDER Car_Owner Propert_Owner  CHILDREN  Annual_income  \
0     5008827      M         Y             Y         0       180000.0   
1     5009744      F         Y             N         0       315000.0   
2     5009746      F         Y             N         0       315000.0   
3     5009749      F         Y             N         0            NaN   
4     5009752      F         Y             N         0       315000.0   
...       ...    ...       ...           ...       ...            ...   
1543  5028645      F         N             Y         0            NaN   
1544  5023655      F         N             N         0       225000.0   
1545  5115992      M         Y             Y         2       180000.0   
1546  5118219      M         Y             N         0       270000.0   
1547  5053790      F         Y             Y         0       225000.0   

               Type_Income                      EDUCATION  \
0                Pensioner      

In [5]:
 df.isnull().sum()

Ind_ID                0
GENDER                7
Car_Owner             0
Propert_Owner         0
CHILDREN              0
Annual_income        23
Type_Income           0
EDUCATION             0
Marital_status        0
Housing_type          0
Birthday_count       22
Employed_days         0
Mobile_phone          0
Work_Phone            0
Phone                 0
EMAIL_ID              0
Type_Occupation     488
Family_Members        0
Unnamed: 18        1548
label                 0
dtype: int64

In [7]:
# Drop the empty unnamed column and unecessary coloumn
df.drop('Unnamed: 18', axis=1, inplace=True, errors='ignore')
df.drop("Ind_ID", axis=1, inplace=True)

In [8]:
 #imputation
 df['GENDER'] =df['GENDER'].fillna('Unknown')
 df['Type_Occupation'] = df['Type_Occupation'].fillna('Unknown')
 df['Birthday_count'] = df['Birthday_count'].fillna('0')
 df['Annual_income'] = df['Annual_income'].fillna(df['Annual_income'].median())

 #feature engineering
 df['Age'] = (-1 * df['Birthday_count'].astype(float)) // 365
 df['Years_Employed'] = (-1 * df['Employed_days'].replace(365243, 0)) // 365
 df.drop(['Birthday_count', 'Employed_days'], axis=1, inplace=True)

In [9]:
 X = pd.get_dummies(df.drop('label', axis=1), drop_first=True)
 y = df['label']

In [34]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE


# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE to balance the dataset
X_res, y_res = SMOTE(random_state=42).fit_resample(X_scaled, y)

# Split the balanced data
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Logistic Regression model
lr = LogisticRegression(max_iter=3000)

# Apply RFE to select 30 best features
rfe = RFE(lr, n_features_to_select=30)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

# Train on selected features
lr.fit(X_train_rfe, y_train)
y_pred = lr.predict(X_test_rfe)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6436363636363637
Confusion Matrix:
 [[167 109]
 [ 87 187]]
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.61      0.63       276
           1       0.63      0.68      0.66       274

    accuracy                           0.64       550
   macro avg       0.64      0.64      0.64       550
weighted avg       0.64      0.64      0.64       550



In [40]:
# Transform your full scaled feature set using RFE
X_scaled_rfe = rfe.transform(X_scaled)

# Now you can safely use this for predictions
df['Churn_Probability'] = lr.predict_proba(X_scaled_rfe)[:, 1]
df.to_csv('model_output.csv', index=False)

In [39]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load prediction results with churn probabilities
df = pd.read_csv("model_output.csv")  # Make sure it includes 'Churn_Probability' and all cleaned features

# 🎛 Sidebar Filters
st.sidebar.header("🔧 Filter Options")
threshold = st.sidebar.slider("Churn Risk Threshold", 0.0, 1.0, 0.5, 0.05)
income_type = st.sidebar.selectbox("Income Type", sorted(df['Type_Income'].dropna().unique()))
education = st.sidebar.selectbox("Education Level", sorted(df['EDUCATION'].dropna().unique()))
housing = st.sidebar.selectbox("Housing Type", sorted(df['Housing_type'].dropna().unique()))

# 🏷️ Dashboard Title
st.title("📊 Churn Prediction Dashboard")

# 🔍 Latest Model Metrics (From your classification report)
st.subheader("🚀 Model Performance")
st.metric("Accuracy", "64.36%")
st.metric("Recall (Churn)", "68%")
st.metric("F1 Score (Churn)", "0.66")

# 📉 Confusion Matrix Visualization
st.subheader("📉 Confusion Matrix")
conf_matrix = [[167, 109], [87, 187]]  # Updated values
fig, ax = plt.subplots()
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
st.pyplot(fig)

# 📌 Feature Importance (Optional — you can update with real model coefficients later)
features = ['Age', 'Years_Employed', 'Annual_income', 'Family_Members']
importances = [0.21, 0.17, 0.19, 0.14]  # Placeholder values
st.subheader("📌 Feature Importance")
fig2, ax2 = plt.subplots()
ax2.barh(features, importances, color='skyblue')
ax2.set_xlabel("Importance Score")
st.pyplot(fig2)

# 🧠 Customer-Level Predictions Table
st.subheader("🧠 High-Risk Customers")
filtered_df = df[
    (df['Churn_Probability'] >= threshold) &
    (df['Type_Income'] == income_type) &
    (df['EDUCATION'] == education) &
    (df['Housing_type'] == housing)
]
st.dataframe(filtered_df[[
    'GENDER', 'Age', 'Years_Employed', 'Annual_income', 'Family_Members',
    'Type_Income', 'EDUCATION', 'Housing_type', 'Churn_Probability'
]].sort_values(by='Churn_Probability', ascending=False).style.highlight_max(axis=0))

# Footer
st.markdown("---")
st.markdown("Created by Himani • Updated with Logistic Regression + SMOTE Results")


2025-07-10 16:37:14.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.107 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 16:37:14.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()